In [ ]:
import stat_arb
from stat_arb.model.data.simulated_data_handler import SimulatedDataHandler
import matplotlib.pyplot as plt
import datetime as dt
import statsmodels.api as sm
import pandas as pd

plt.style.use("seaborn-v0_8")

In [ ]:
tickers = ["^SPX", "DUMMY"]
start = dt.date(2020, 1, 1)
end = dt.date(2025, 1, 8)
live = dt.date(2025, 1, 6)

data = SimulatedDataHandler(tickers, start, end, corr=0.9)

ts = data.get_close_prices()

ts.plot();

In [ ]:
from statsmodels.regression.rolling import RollingOLS

ts = sm.add_constant(ts)

In [ ]:
ts

In [ ]:
ols = RollingOLS(ts[["^SPX"]], ts[["const", "DUMMY"]], window=252)
ols = ols.fit()


In [ ]:
ols.params.plot()

In [ ]:
fig, ax = plt.subplots(2,1)
ax[0].plot(ts[tickers])
ax[1].plot(ols.params)
ax[0].legend(loc="upper right")


In [ ]:
regression = ols.params["const"] * ts["const"] + ols.params["DUMMY"] * ts["DUMMY"]

df =pd.concat((ts, regression), axis=1)
df.rename(columns={0:"Regression"}, inplace=True)
df["resids"] = df["^SPX"] - df["Regression"]
df


In [ ]:
df["resids"].plot();

In [ ]:
from statsmodels.tsa.stattools import coint

cadf = coint(ts["^SPX"], ts["DUMMY"], return_results=True)
cadf

In [ ]:
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm

ols = sm.OLS(ts["^SPX"], sm.add_constant(ts["DUMMY"]))
ols = ols.fit()
adfuller(ols.resid)

In [ ]:
from stat_arb.model.bivariate_engle_granger import BivariateEngleGranger
from stat_arb.model.data.data_handler_enum import DataHandlerEnum
from stat_arb.model.trading_strategy import StrategyEnum
from stat_arb.model.trading_strategy import ToyStrategyInputs 
import datetime as dt

ticker_a = "MSFT"
ticker_b = "AMZN"
start = dt.date(2020, 1, 1)
end = dt.date(2025, 1, 8)
live = dt.date(2025, 1, 6)
source = DataHandlerEnum.SIMULATED
model = BivariateEngleGranger(ticker_a, ticker_b, start, end, live, source)


strategy_type = StrategyEnum.ToyStrategy
strategy_inputs = ToyStrategyInputs(1,0)

df = model.run(strategy_type, strategy_inputs)

In [ ]:
df.get_backtest()

In [ ]:
model.get_data()

model.get_residual().plot()

In [ ]:
import pandas as pd

link = r"https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

wiki = pd.read_html(link)

In [ ]:
wiki[0]["Symbol"].to_csv("tickers_20250418.csv", index=False)

In [ ]:
from matplotlib import legend
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd

df = px.data.stocks()

fig1 = px.line(df, x="date", y=["GOOG", "AAPL"], title="GOOG")
fig2 = px.line(df, x="date", y=["GOOG", "AAPL"], title="AAPL")

fig = make_subplots(rows=2, cols=1, subplot_titles=("GOOG", "AAPL"))

fig.add_traces(fig1.data, rows=1, cols=1)
fig.add_traces(fig2.data, rows=2, cols=1)

fig.update_layout(title="Stock Prices", height=900, width=900)



In [ ]:
from matplotlib import legend
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd

df = px.data.stocks()

fig1 = px.line(df, x="date", y=["GOOG", "AAPL"], title="GOOG")
fig2 = px.line(df, x="date", y=["GOOG", "AAPL"], title="AAPL")

fig = make_subplots(rows=2, cols=1, subplot_titles=("GOOG", "AAPL"))

fig.add_traces(fig1.data, rows=1, cols=1)
fig.add_traces(fig2.data, rows=2, cols=1)

fig.update_layout(title="Stock Prices", height=900, width=900)


In [ ]:
import numpy as np
import pandas as pd

x = np.random.normal(size=[100])
x = x.cumsum()

df = pd.DataFrame(x, columns=["Residual"])

def enter_signal(a, enter):
    if a < (-enter):
        return -1  # enter long
    elif a > (enter):
        return 1  # enter short
    else: 
        return 0
    
def exit_signal(a, exit):
    if a > exit:
        return -1 #"exit long"
    elif a < (-exit):
        return 1 #"exit short"
    else:
        return 0
    
def signal(a, enter, exit):
    if a < -enter:
        return 1
    elif a > enter:
        return -1
    elif abs(a) < exit:
        return 0
    else:
        return None

enter = 1
exit = 0.2

df["LongEntrySignal"] = df["Residual"] < -enter
df["ShortEntrySignal"] = df["Residual"] > enter
df["LongExitSignal"] = df["Residual"] > -exit
df["ShortExitSignal"] = df["Residual"] < exit

#df["ExitSignal"] = df["Residual"] < abs(exit)
# df["ExitSignal"] = df["Residual"].apply(lambda x: exit_signal(x, 0))
# df["Signal"] = df["Residual"].apply(lambda x: signal(x, 2, 0.0))

current_signal = 0
signals = []

for index, row in df.iterrows():
    if current_signal == 0:
        if row["LongEntrySignal"]:
            new_signal = 1
        elif row["ShortEntrySignal"]:
            new_signal = -1
        else:
            new_signal = 0
    elif current_signal == 1:
        if row["ShortEntrySignal"]:
            new_signal = -1
        elif row["LongExitSignal"]:
            new_signal = 0
        else:
            new_signal = 1
    else:  # current_signal == -1
        if row["LongEntrySignal"]:
            new_signal = 1
        elif row["ShortExitSignal"]:
            new_signal = 0
        else:
            new_signal = -1

    current_signal = new_signal
    signals.append(new_signal)

df["Signal"] = signals

#df["Signal"] = df["Signal"].ffill().fillna(0)

df.plot()

In [ ]:
import numpy as np
import pandas as pd

x = np.random.normal(size=[1000])
x = x.cumsum()

df = pd.DataFrame(x, columns=["Residual"])

window = 90

df["Z-Score"] = (df["Residual"] - df["Residual"].rolling(window).mean()) / df["Residual"].rolling(window).std()

enter = 1
exit = 0.2

df["LongEntrySignal"] = df["Z-Score"] < -enter
df["ShortEntrySignal"] = df["Z-Score"] > enter
df["LongExitSignal"] = df["Z-Score"] > -exit
df["ShortExitSignal"] = df["Z-Score"] < exit


current_signal = 0
signals = []

for index, row in df.iterrows():
    if current_signal == 0:
        if row["LongEntrySignal"]:
            new_signal = 1
        elif row["ShortEntrySignal"]:
            new_signal = -1
        else:
            new_signal = 0
    elif current_signal == 1:
        if row["ShortEntrySignal"]:
            new_signal = -1
        elif row["LongExitSignal"]:
            new_signal = 0
        else:
            new_signal = 1
    else:  # current_signal == -1
        if row["LongEntrySignal"]:
            new_signal = 1
        elif row["ShortExitSignal"]:
            new_signal = 0
        else:
            new_signal = -1

    current_signal = new_signal
    signals.append(new_signal)

df["Signal"] = signals

#df["Signal"] = df["Signal"].ffill().fillna(0)

df.plot()

In [ ]:
pd.set_option('display.max_rows', 1000)

df

In [ ]:
from stat_arb.model.bivariate_engle_granger import BivariateEngleGranger
from stat_arb.model.data.data_handler_enum import DataHandlerEnum
import plotly.express as px
import datetime as dt

ticker_a = "MSFT"
ticker_b = "AMZN"
start = dt.date(2020, 1, 1)
end = dt.date(2025, 1, 8)
live = dt.date(2025, 1, 6)
source = DataHandlerEnum.SIMULATED
model = BivariateEngleGranger(ticker_a, ticker_b, start, end, live, source)
ts = model.get_close_prices()